In [1]:
import requests
import json
from collections import defaultdict

def fetch_and_parse_data(url):
    response = requests.get(url)
    data = json.loads(response.text)
    return data

def filter_and_extract_data(parsed_data, filtered_data):
    for card in parsed_data:
        if card['type_code'] in ['skill', 'asset', 'event']:
            card_key = card['code']
            if card_key not in filtered_data:
                filtered_data[card_key] = {
                    'name': card['name'],
                    'quantity': card['quantity'],
                    'code': card['code'],
                    'xp': card.get('xp', 0)
                }
            else:
                filtered_data[card_key]['quantity'] += card['quantity']
    return filtered_data

def fetch_and_filter_data(packs):
    filtered_data = {}
    base_url = 'https://ko.arkhamdb.com/api/public/cards/'
    for pack in packs:
        url = base_url + pack
        parsed_data = fetch_and_parse_data(url)
        filtered_data = filter_and_extract_data(parsed_data, filtered_data)
    return filtered_data

def fetch_decks(deck_ids):
    deck_data = defaultdict(int)
    for deck_id in deck_ids:
        base_url = 'https://ko.arkhamdb.com/api/public/deck/'
        url = base_url + deck_id
        individual_deck_data = fetch_and_parse_data(url)
        for card, quantity in individual_deck_data['slots'].items():
            deck_data[card] += quantity
    return deck_data

def fetch_card_info(card_code):
    base_url = 'https://ko.arkhamdb.com/api/public/card/'
    url = base_url + card_code
    card_data = fetch_and_parse_data(url)
    return card_data

def compare_cards(filtered_data, deck_data):
    missing_cards = {}
    for card, quantity in deck_data.items():
        if card[0:3] == '010':
            card = '015' + card[3:]
        if card not in filtered_data or filtered_data[card]['quantity'] < quantity:
            card_info = fetch_card_info(card)
            if card_info.get('type_code','') in ['skill', 'asset', 'event']:
                missing_cards[card_info['name']] = quantity - filtered_data.get(card,{}).get('quantity', 0)
                print(card, card_info['name'],'는 ',quantity,'장이 필요하지만, ',filtered_data.get(card,{}).get('quantity', 0),'장만 있습니다.')
    return missing_cards

In [2]:
packs = ['rcore', 'dwl', 'tmm', 'tece', 'uau', 'bota', 'wda', 'litas']  # replace with actual pack names
deck_ids = ['2978726', '2978725', '2978724']  # replace with actual deck id

filtered_data = fetch_and_filter_data(packs)
deck_data = fetch_decks(deck_ids)

missing_cards = compare_cards(filtered_data, deck_data)

01517 체력 단련 는  3 장이 필요하지만,  2 장만 있습니다.
01523 재빨리 피하다 는  4 장이 필요하지만,  2 장만 있습니다.
01524 다이너마이트 폭발 는  3 장이 필요하지만,  2 장만 있습니다.
01550 도피 는  4 장이 필요하지만,  2 장만 있습니다.
01565 보호의 진 는  4 장이 필요하지만,  2 장만 있습니다.
01593 뜻밖의 용기 는  6 장이 필요하지만,  4 장만 있습니다.
02026 대박 아니면 쪽박 는  4 장이 필요하지만,  2 장만 있습니다.
01533 밀란 크리스토퍼 박사 는  3 장이 필요하지만,  2 장만 있습니다.
